In [2]:
import pandas as pd

df = pd.read_csv("/home/lupusruber/music_analytics/data/events.csv", encoding="utf-8")
df.drop(["Unnamed: 0"], inplace=True, axis=1)

In [3]:
transform_string = (
    lambda x: "".join([("_" + i if i.isupper() else i) for i in x]).strip().lower()
)

In [4]:
df.columns = list(map(transform_string, df.columns.tolist()))
df = df.rename({"ts": "timestamp"}, axis=1)

In [5]:
df.head(1)

,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.373463,35.380939,9783,Parker,Haley,F,1616086800000


In [7]:
df.groupby("session_id").count().reset_index()

,session_id,artist,song,duration,timestamp,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,10,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,12,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
3,28,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,31,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14657,55788,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
14658,55805,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
14659,55848,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
14660,55882,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [8]:
df.groupby("gender")["gender"].count() / len(df)

gender
F    0.47126
M    0.52874
Name: gender, dtype: float64

In [9]:
df.query("user_id==748 and item_in_session==30").sort_values(
    "timestamp", ascending=True
).head(10)

,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
650,Irma Thomas,The Same Love That Made Me Laugh [LP Version],195.36934,1616087608000,747,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000
80143,Chris Vickery,Jokers and Thieves,191.71220,1616302962000,49237,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000


In [10]:
result = (
    df.groupby("session_id")
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# result['count'].sum() == n_rows
result

,session_id,count
7220,37432,219
7958,38794,216
13931,51602,199
4659,30891,197
8117,39079,178
...,...,...
4605,30649,1
4637,30787,1
4635,30784,1
4634,30781,1


In [11]:
df.query("session_id == 37432")[
    ["artist", "song", "duration", "item_in_session", "user_id"]
]["user_id"].unique()

array([17692])

In [12]:
session_and_user = (
    df.groupby(["session_id", "user_id"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# df.groupby(['sessionId', 'userId']).size().reset_index(name='count')['count].sum() == n_rows
session_and_user

,session_id,user_id,count
7220,37432,17692,219
7958,38794,17940,216
13931,51602,19676,199
4659,30891,16304,197
8117,39079,17780,178
...,...,...,...
4605,30649,1903,1
4637,30787,10466,1
4635,30784,13727,1
4634,30781,17442,1


In [13]:
session_and_user.groupby("user_id").size().reset_index(name="count").sort_values(
    "count", ascending=False
)

,user_id,count
5726,13473,17
3371,8010,16
121,288,15
3119,7357,13
3783,8943,12
...,...,...
8439,19986,1
8425,19958,1
8426,19959,1
8428,19964,1


In [14]:
session_and_user.sort_values("user_id")

,session_id,user_id,count
4132,28898,3,3
10935,44377,8,1
6093,35076,8,3
0,9,10,1
10626,43735,11,5
...,...,...,...
14085,52141,19996,1
13523,50407,19997,5
2870,19996,19997,1
7136,37261,20001,37


In [ ]:
df["user_id"].unique().shape[0] == df["session_id"].unique().shape[0]

False

In [ ]:
df.query("user_id == 19997").groupby("session_id").aggregate(lambda x: x)[
    "itemInSession"
]

sessionId
19996                  0
50407    [0, 1, 3, 5, 6]
Name: itemInSession, dtype: object

In [5]:
auth_events = pd.read_csv("/home/lupusruber/music_analytics/data/auth_events.csv")
auth_events.head()

,Unnamed: 0,ts,sessionId,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration,success
0,0,1616086832000,36313,paid,18,Torrance,90504,CA,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:30....,-118.330794,33.867257,17996.0,Key,Timothy,M,1.616087e+12,True
1,1,1616086871000,32671,paid,1,York,36925,AL,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-88.183591,32.432254,5102.0,Reed,Oliver,M,1.616087e+12,True
2,2,1616086960000,747,paid,28,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748.0,Ramirez,Kyra,F,1.616087e+12,True
3,3,1616087004000,35,free,4,Altamont,12009,NY,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) G...,-74.035542,42.697566,36.0,Shaw,Katelyn,F,1.616087e+12,True
4,4,1616087050000,34007,paid,8,Matthews,28105,NC,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-80.713708,35.115597,2582.0,Ashley,Ava,F,1.616087e+12,True


In [18]:
auth_events.query("success != True").head(3)

,Unnamed: 0,ts,sessionId,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration,success
44,44,1616090567000,38169,paid,1,San Antonio,78236,TX,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",-98.618734,29.391283,NaN,NaN,NaN,NaN,NaN,False
45,45,1616090568000,38169,paid,2,San Antonio,78236,TX,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",-98.618734,29.391283,NaN,NaN,NaN,NaN,NaN,False
62,62,1616092648000,38122,paid,12,Rancho Cordova,95670,CA,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-121.280327,38.604268,NaN,NaN,NaN,NaN,NaN,False


In [3]:
page_view = pd.read_csv("/home/lupusruber/music_analytics/data/page_view_events.csv")
page_view.drop(["Unnamed: 0", "userAgent"], inplace=True, axis=1)
page_view.head()

,ts,sessionId,page,auth,method,status,level,itemInSession,city,zip,...,lon,lat,userId,lastName,firstName,gender,registration,artist,song,duration
0,1616086802000,9782,NextSong,Logged In,PUT,200,free,2,Flagstaff,86004,...,-111.373463,35.380939,9783.0,Parker,Haley,F,1.616087e+12,Ayo,Welcome Into My World,264.56772
1,1616086803000,33160,NextSong,Logged In,PUT,200,paid,6,Cincinnati,45238,...,-84.610282,39.107658,16205.0,Smith,Jordan,F,1.616087e+12,Gershon Kingsley,Trumansburg Whistle,180.08770
2,1616086804000,16311,NextSong,Logged In,PUT,200,paid,12,Pecos,79772,...,-103.627309,31.422363,16312.0,Flores,Kylen,M,1.616087e+12,AUDIO.3,SPONT.AN,241.44934
3,1616086805000,6939,NextSong,Logged In,PUT,200,free,0,La Puente,91744,...,-117.936470,34.031415,6940.0,Nelson,Caedyn,F,1.616087e+12,Mario Piu,Communication,430.54974
4,1616086806000,1095,NextSong,Logged In,PUT,200,paid,7,Morrisonville,62546,...,-89.445923,39.431455,1096.0,Gonzalez,Emma,F,1.616087e+12,Boyz II Men,In My Life,151.58812


In [19]:
page_view["page"].unique()

array(['NextSong', 'Upgrade', 'Submit Upgrade', 'Settings', 'Home',
       'Login', 'Logout', 'Downgrade', 'About', 'Help', 'Error',
       'Save Settings', 'Submit Downgrade', 'Cancel',
       'Cancellation Confirmation'], dtype=object)

In [15]:
page_view.columns

Index(['Unnamed: 0', 'ts', 'sessionId', 'page', 'auth', 'method', 'status',
       'level', 'itemInSession', 'city', 'zip', 'state', 'userAgent', 'lon',
       'lat', 'userId', 'lastName', 'firstName', 'gender', 'registration',
       'artist', 'song', 'duration'],
      dtype='object')

In [11]:
status_change = pd.read_csv(
    "/home/lupusruber/music_analytics/data/status_change_events.csv"
)
status_change.head()

,Unnamed: 0,ts,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
0,0,1616086812000,6939,Logged In,free,3,La Puente,91744,CA,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,-117.936470,34.031415,6940,Nelson,Caedyn,F,1616086800000
1,1,1616087702000,295,Logged In,free,2,Warren,44485,OH,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",-80.847586,41.240357,296,Elliott,Ayla,F,1616086800000
2,2,1616090299000,30303,Logged In,free,3,Falls City,78113,TX,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) G...,-98.240425,28.867033,5414,Rodriguez,Emilee,F,1616086800000
3,3,1616091105000,23312,Logged In,free,3,Dodge City,67801,KS,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-100.057186,37.723603,3832,Hubbard,Kameron,F,1616086800000
4,4,1616091948000,39091,Logged In,free,2,Weirton,26062,WV,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,-80.560680,40.426703,17542,Singh,Gavin,M,1616086800000


In [20]:
status_change.columns

Index(['Unnamed: 0', 'ts', 'sessionId', 'auth', 'level', 'itemInSession',
       'city', 'zip', 'state', 'userAgent', 'lon', 'lat', 'userId', 'lastName',
       'firstName', 'gender', 'registration'],
      dtype='object')

In [14]:
import pandas as pd

df = pd.read_csv(
    "/home/lupusruber/music_analytics/data/listen_events.csv", encoding="utf-8"
)
df.drop(["Unnamed: 0", "userAgent"], inplace=True, axis=1)
df.head()

,artist,song,duration,ts,sessionId,auth,level,itemInSession,city,zip,state,lon,lat,userId,lastName,firstName,gender,registration
0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,-111.373463,35.380939,9783,Parker,Haley,F,1616086800000
1,Gershon Kingsley,Trumansburg Whistle,180.08770,1616086803000,33160,Logged In,paid,6,Cincinnati,45238,OH,-84.610282,39.107658,16205,Smith,Jordan,F,1616086800000
2,AUDIO.3,SPONT.AN,241.44934,1616086804000,16311,Logged In,paid,12,Pecos,79772,TX,-103.627309,31.422363,16312,Flores,Kylen,M,1616086800000
3,Mario Piu,Communication,430.54974,1616086805000,6939,Logged In,free,0,La Puente,91744,CA,-117.936470,34.031415,6940,Nelson,Caedyn,F,1616086800000
4,Boyz II Men,In My Life,151.58812,1616086806000,1095,Logged In,paid,7,Morrisonville,62546,IL,-89.445923,39.431455,1096,Gonzalez,Emma,F,1616086800000
